<a href="https://colab.research.google.com/github/KarelZe/thesis/blob/accuracy/notebooks/4.0b-mb-gbm_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install catboost==1.1
!pip install gcsfs==2022.10.0
!pip install ipywidgets==8.0.2
!pip install numpy==1.23.4
!pip install pandas==1.5.1
!pip install scikit-learn==1.1.3
!pip install fastparquet
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.7 MB 1.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 138 kB 27.5 MB/s 
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.11.0
    Uninstalling fsspec-2022.11.0:
      Successfully uninstalled fsspec-2022.11.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 134 kB 15.6 MB/s 
     |████████████████████████████████| 2.0 MB 67.1 MB/s 
     |████████████████████████████████| 1.6 MB 62.2 MB/s 
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.1
    Uninstalling widgetsnbextension-3.6.1:
      Successfully uninstalled widgetsnbextension-3.6.1
  Attempting uninstall: ipywidgets
    Found exis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.2 MB 14.9 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 31.2 MB 1.5 MB/s 
     |████████████████████████████████| 16.9 MB 58.8 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.4
    Uninstalling numpy-1.23.4:
      Successfully uninstalled numpy-1.23.4
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 15.2 MB/s 
     |████████████████████████████████| 2.0 MB 64.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 15.0 MB/s 
     |████████████████████████████████| 182 kB 74.5 MB/s 
     |████████████████████████████████| 168 kB 74.8 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
     |████████████████████████████████| 168 kB 78.5 MB/s 
     |████████████████████████████████| 166 kB 81.1 MB/s 
     |████████████████████████████████| 166 kB 85.4 MB/s 
     |████████████████████████████████| 162 kB 79.3 MB/s 
     |████████████████████████████████| 162 kB 79.3 MB/s 
     |████████████████████████████████| 158 kB 82.3 MB/s 
     |████████████████████████████████| 157 kB 83.7 MB/s 
     |████████████████████████████████| 157 kB 86.

Run `pip install .` first to install all dependencies.

In [2]:
from catboost import CatBoostClassifier

import gcsfs
import google.auth
from google.colab import auth, output

import numpy as np
import pandas as pd

import os
from pathlib import Path

from sklearn.metrics import accuracy_score

from tqdm.notebook import tqdm
import wandb

from typing import List, Optional

In [3]:
# connect to google cloud storage
auth.authenticate_user()
credentials, _ = google.auth.default()
fs = gcsfs.GCSFileSystem(project="thesis", token=credentials)
fs_prefix = "gs://"

In [4]:
features_classical_size = [
    'TRADE_PRICE', 'bid_ask_size_ratio_ex', 'rel_bid_size_ex',
       'rel_ask_size_ex', 'TRADE_SIZE', 'bid_size_ex', 'ask_size_ex',
       'rel_ask_ex', 'rel_bid_ex', 'BEST_rel_bid', 'BEST_rel_ask',
       'bid_ask_ratio_ex', 'chg_ex_lead', 'chg_ex_lag', 'chg_all_lead',
       'chg_all_lag', 'ask_ex', 'bid_ex', 'BEST_ASK', 'BEST_BID',
       'price_all_lag', 'price_all_lead', 'price_ex_lag', 'price_ex_lead'
]

In [5]:
# see https://wandb.ai/fbv/thesis/runs/kwlaw02g/overview?workspace=user-karelze
# for refs

run = wandb.init(project="thesis",entity="fbv")

dataset = "fbv/thesis/classical_size_features_log_normalized:v0"
artifact = run.use_artifact(dataset)
data_dir = artifact.download()

model = "fbv/thesis/3jkwpy3w_CatBoostClassifier_default_trial_49.cbm:v0"
artifact = run.use_artifact(model)
model_dir = artifact.download()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Downloading large artifact classical_size_features_log_normalized:v0, 2564.04MB. 3 files... 
wandb:   3 of 3 files downloaded.  
Done. 0:1:14.3
wandb: Downloading large artifact 3jkwpy3w_CatBoostClassifier_default_trial_49.cbm:v0, 101.05MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:5.4


In [6]:
X_test = pd.read_parquet(Path(data_dir, "test_set_20.parquet"), engine="fastparquet")

y_test = X_test["buy_sell"]
y_test[y_test<0] = 0
X_test = X_test[features_classical_size]


<ipython-input-6-800bdc36999a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_test[y_test<0] = 0


In [7]:
X_test.head()

,TRADE_PRICE,bid_ask_size_ratio_ex,rel_bid_size_ex,rel_ask_size_ex,TRADE_SIZE,bid_size_ex,ask_size_ex,rel_ask_ex,rel_bid_ex,BEST_rel_bid,...,chg_all_lead,chg_all_lag,ask_ex,bid_ex,BEST_ASK,BEST_BID,price_all_lag,price_all_lead,price_ex_lag,price_ex_lead
39342171,-0.886605,-0.999944,-0.999978,-0.999959,-0.920668,-0.397940,-0.568327,-0.450753,0.450753,-0.222609,...,0.979614,0.887864,-0.869878,-0.883935,-0.922989,-0.883936,-0.917824,-0.921640,-0.897066,-0.875702
39342172,-0.398830,-0.999980,-0.999660,-0.999773,-1.000000,-0.879588,-0.879588,-0.449826,0.449826,-0.221914,...,0.979609,0.887724,-0.387113,-0.403539,-0.637270,-0.403545,-0.511138,-0.615242,-0.380144,-0.521084
39342173,-0.060248,-0.999980,-0.999660,-0.999773,-1.000000,-0.879588,-0.879588,-0.450689,0.450689,-0.222561,...,0.979589,0.887492,-0.031559,-0.059053,-0.426840,-0.059063,-0.248786,-0.402171,-0.057769,-0.080426
39342174,-0.675788,-0.999980,-0.999966,-0.999977,-1.000000,-0.583443,-0.583443,-0.448436,0.448436,-0.220290,...,0.979647,0.887998,-0.673685,-0.709687,-0.811995,-0.709690,-0.802774,-0.914062,-1.000000,-1.000000
39342175,-0.727221,-0.999980,-0.999966,-0.999977,-1.000000,-0.583443,-0.583443,-0.450753,0.450753,-0.222609,...,0.979636,0.887758,-0.700498,-0.724674,-0.822743,-0.724677,-0.747425,-0.913006,-1.000000,-1.000000


## CatBoost Baseline 🐈‍⬛

In [8]:
model = CatBoostClassifier()
model.load_model(fname=Path(model_dir, "3jkwpy3w_CatBoostClassifier_default_trial_49.cbm"))

In [9]:
acc = model.score(X_test, y_test)

print(acc)

/usr/local/lib/python3.8/dist-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


0.7262843180440935


### Robustness Baseline🥊

In [10]:
# load default data to use unscaled version with all possible columns
test_orig = pd.read_parquet(
    f"gs://thesis-bucket-option-trade-classification/data/preprocessed/test_set_extended_20.parquet",
    engine="fastparquet",
)

In [11]:
test_orig.dtypes

QUOTE_DATETIME    datetime64[ns]
ROOT                      object
EXPIRATION        datetime64[ns]
STRK_PRC                 float32
OPTION_TYPE             category
TRADE_SIZE                 int64
TRADE_PRICE              float32
BEST_BID                 float32
BEST_ASK                 float32
ask_ex                   float32
bid_ex                   float32
bid_size_ex              float32
ask_size_ex              float32
price_all_lead           float32
price_all_lag            float32
optionid                 float32
day_vol                  float32
price_ex_lead            float32
price_ex_lag             float32
issue_type                object
myn                      float32
buy_sell                    int8
dtype: object

In [12]:
# Copy unscaled columns
X_print = test_orig.copy()
X_print["buy_sell"] = (X_print["buy_sell"] > 0).astype(int)

# add baseline results
X_print["rule"] = "Baseline"
X_print["buy_sell_predicted"] = model.predict(X_test)

# prepare columns for printing
X_print["ttm"] = (
    X_print["EXPIRATION"].dt.to_period("M")
    - X_print["QUOTE_DATETIME"].dt.to_period("M")
).apply(lambda x: x.n)
X_print["year"] = X_print["QUOTE_DATETIME"].dt.year

bins_tradesize = [-np.inf, 1, 3, 5, 11, np.inf]
trade_size_labels = ["(0,1]", "(1,3]", "(3,5]", "(5,11]", ">11"]
X_print["TRADE_SIZE_binned"] = pd.cut(
    X_print["TRADE_SIZE"], bins_tradesize, labels=trade_size_labels
)

bins_years = [2004, 2007, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
year_labels = [
    "2005-2007",
    "2008-2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
]
X_print["year_binned"] = pd.cut(X_print["year"], bins_years, labels=year_labels)

bins_ttm = [-np.inf, 1, 2, 3, 6, 12, np.inf]
ttm_labels = [
    "ttm <= 1 month",
    "ttm (1-2] month",
    "ttm (2-3] month",
    "ttm (3-6] month",
    "ttm (6-12] month",
    "ttm > 12 month",
]
X_print["ttm_binned"] = pd.cut(X_print["ttm"], bins_ttm, labels=ttm_labels)


bins_myn = [-np.inf, 0.7, 0.9, 1.1, 1.3, np.inf]
myn_labels = [
    "mny <=0.7",
    "mny (0.7-0.9]",
    "mny (0.9-1.1]",
    "mny (1.1-1.3]",
    "mny > 1.3",
]
X_print["myn_binned"] = pd.cut(X_print["myn"], bins_myn, labels=myn_labels)

X_print["issue_type_binned"] = X_print["issue_type"].replace(
    {"0": 'Stock options', 'A': 'Index options', '7': 'Others',
     'F': 'Others', '%': 'Others', ' ': 'Others'})


# TODO: time from previous trade; same underlying or any?


/usr/local/lib/python3.8/dist-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


In [13]:
X_print.head()

,QUOTE_DATETIME,ROOT,EXPIRATION,STRK_PRC,OPTION_TYPE,TRADE_SIZE,TRADE_PRICE,BEST_BID,BEST_ASK,ask_ex,...,buy_sell,rule,buy_sell_predicted,ttm,year,TRADE_SIZE_binned,year_binned,ttm_binned,myn_binned,issue_type_binned
39342171,2015-11-06 09:30:00,EWZ,2015-12-18,22.0,P,2,0.520000,0.520000,0.600000,0.600000,...,0,Baseline,0,1,2015,"(1,3]",2015,ttm <= 1 month,mny (0.9-1.1],Others
39342172,2015-11-06 09:30:00,TSLA,2015-11-27,230.0,C,1,7.820000,7.600000,8.150000,8.150000,...,0,Baseline,0,0,2015,"(0,1]",2015,ttm <= 1 month,mny (0.9-1.1],Stock options
39342173,2015-11-06 09:30:00,TSLA,2017-01-20,260.0,C,1,28.889999,28.799999,32.049999,32.049999,...,0,Baseline,0,14,2015,"(0,1]",2015,ttm > 12 month,mny (0.7-0.9],Stock options
39342174,2015-11-06 09:30:00,VB,2015-11-20,115.0,C,1,2.250000,1.850000,2.150000,2.250000,...,1,Baseline,1,0,2015,"(0,1]",2015,ttm <= 1 month,mny (0.9-1.1],Others
39342175,2015-11-06 09:30:00,VB,2015-12-18,117.0,C,1,1.700000,1.700000,1.950000,1.950000,...,0,Baseline,0,1,2015,"(0,1]",2015,ttm <= 1 month,mny (0.9-1.1],Others


In [14]:
def check_robustness(criterion: str = "year_binned") -> pd.DataFrame:
    """
    Check robustness of rules by calculating the accuracy for a given
    criterion and rules.

    Example:
    rule		Baseline
    TRADE_SIZE_binned
    (0,1]	  0.710966
    (1,3]	  0.717664
    (3,5]	  0.715195
    (5,11]	0.699428
    >11	  	0.688348

    Args:
        criterion (str, optional): criterion to check robustness for.
        Defaults to "year_binned".

    Returns:
        pd.DataFrame: DataFrame with accuracy of rules. Rule in columns and
        criterion values in rows.
    """

    # fill others randomly with equal weight for every class.
    X_print["buy_sell_predicted"] = X_print["buy_sell_predicted"].map(
        lambda l: l if not np.isnan(l) else np.random.choice([0, 1])
    )

    # cuculate average over columns if multiple subsets are combined
    results = (
        X_print.groupby(["rule", criterion])[["buy_sell", "buy_sell_predicted"]]
        .apply(lambda x: accuracy_score(x["buy_sell"], x["buy_sell_predicted"]))
        .unstack(level=0)
        .assign(avg=lambda x: x.mean(axis=1))
    )
    return results


In [15]:
X_print.head()


,QUOTE_DATETIME,ROOT,EXPIRATION,STRK_PRC,OPTION_TYPE,TRADE_SIZE,TRADE_PRICE,BEST_BID,BEST_ASK,ask_ex,...,buy_sell,rule,buy_sell_predicted,ttm,year,TRADE_SIZE_binned,year_binned,ttm_binned,myn_binned,issue_type_binned
39342171,2015-11-06 09:30:00,EWZ,2015-12-18,22.0,P,2,0.520000,0.520000,0.600000,0.600000,...,0,Baseline,0,1,2015,"(1,3]",2015,ttm <= 1 month,mny (0.9-1.1],Others
39342172,2015-11-06 09:30:00,TSLA,2015-11-27,230.0,C,1,7.820000,7.600000,8.150000,8.150000,...,0,Baseline,0,0,2015,"(0,1]",2015,ttm <= 1 month,mny (0.9-1.1],Stock options
39342173,2015-11-06 09:30:00,TSLA,2017-01-20,260.0,C,1,28.889999,28.799999,32.049999,32.049999,...,0,Baseline,0,14,2015,"(0,1]",2015,ttm > 12 month,mny (0.7-0.9],Stock options
39342174,2015-11-06 09:30:00,VB,2015-11-20,115.0,C,1,2.250000,1.850000,2.150000,2.250000,...,1,Baseline,1,0,2015,"(0,1]",2015,ttm <= 1 month,mny (0.9-1.1],Others
39342175,2015-11-06 09:30:00,VB,2015-12-18,117.0,C,1,1.700000,1.700000,1.950000,1.950000,...,0,Baseline,0,1,2015,"(0,1]",2015,ttm <= 1 month,mny (0.9-1.1],Others


In [16]:
check_robustness("year_binned")


rule,Baseline,avg
year_binned,,
2015,0.695816,0.695816
2016,0.728504,0.728504
2017,0.731394,0.731394


In [17]:
check_robustness("OPTION_TYPE")


rule,Baseline,avg
OPTION_TYPE,,
C,0.721599,0.721599
P,0.731635,0.731635


In [18]:
check_robustness("issue_type_binned")

rule,Baseline,avg
issue_type_binned,,
Index options,0.577778,0.577778
Others,0.766301,0.766301
Stock options,0.712416,0.712416


In [19]:
check_robustness("TRADE_SIZE_binned")


rule,Baseline,avg
TRADE_SIZE_binned,,
"(0,1]",0.728618,0.728618
"(1,3]",0.733653,0.733653
"(3,5]",0.731215,0.731215
"(5,11]",0.719388,0.719388
>11,0.714786,0.714786


In [20]:
check_robustness("ttm_binned")


rule,Baseline,avg
ttm_binned,,
ttm <= 1 month,0.730221,0.730221
ttm (1-2] month,0.730788,0.730788
ttm (2-3] month,0.721475,0.721475
ttm (3-6] month,0.716497,0.716497
ttm (6-12] month,0.714440,0.714440
ttm > 12 month,0.691314,0.691314


In [21]:
check_robustness("myn_binned")


rule,Baseline,avg
myn_binned,,
mny <=0.7,0.717504,0.717504
mny (0.7-0.9],0.743626,0.743626
mny (0.9-1.1],0.731846,0.731846
mny (1.1-1.3],0.674266,0.674266
mny > 1.3,0.642889,0.642889
